In [1]:
import pandas as pd
from SentiCR.SentiCR import SentiCR

SentiCR/data/so-train.pkl


## Preprocessing

In [5]:
df_original = pd.read_xml('test.xml')
df = df_original[4:].drop(['team_domain', 'channel_name', 'start_date', 'end_date'], axis=1)

### SentiCR preprocessing

In [14]:
import numpy as np
from sklearn.metrics import classification_report
import time
sentiment_analyzer=SentiCR()
def Extract(lst):
    return [item[0] for item in lst]
def predict_sentiCR(df, column = 'first_message'):
    begin=time.time()
    sentences=df[column]
    pred=[]
    for sent in sentences:
        score=sentiment_analyzer.get_sentiment_polarity(sent)
        pred.append(score)
    
    end=time.time()
    print('Prediction used {:.2f} seconds'.format(end-begin))
    return pred

StackOverflow Reading data from oracle..
Training classifier model..


/home/tom/.pyenv/versions/3.8.15/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/tom/.pyenv/versions/3.8.15/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ani', 'continu', 'deleg', 'doe', 'doubl', 'dure', 'els', 'endwhil', 'extend', 'implement', 'includ', 'interfac', 'namespac', 'nativ', 'nowwhil', 'onc', 'ourselv', 'overrid', 'packag', 'privat', 'protect', 'rais', 'readon', 'requir', 'sign', 'synchron', 'themselv', 'tri', 'veri', 'yourselv'] not in stop_words.
  warnings.warn(


Training used 7.65 seconds


In [36]:
# df_sentiCR_closure = pd.read_csv('df_senti4SD_closure.csv', delimiter=';', index_col="ID")
# pred_closure_sentiCR = predict_sentiCR(df_sentiCR_closure, 'Text')
# df_sentiCR_closure['sentiCR'] = Extract(pred_closure_sentiCR)
# df_sentiCR_closure['sentiCR'] = df_sentiCR_closure['sentiCR'].replace({2: -1})
# df_sentiCR_closure.to_csv('predictions_sentiCR_closure.csv')
# df_sentiCR_closure
df_sentiCR_closure = pd.read_csv('predictions_sentiCR_closure.csv', index_col='ID')
df_full_closure = pd.read_csv('full_clojur_df.csv', index_col='Unnamed: 0')
df_full_closure['sentiCR'] = df_sentiCR_closure['sentiCR']
df_full_closure

,conversation_id,ts,user,text,sentiCR
1,1.0,2019-11-01T19:36:28.998000,Ethaniel,It's generally always possible to write map an...,0
2,2.0,2019-11-01T19:50:03.330000,Abbigail,Some languages it's far more unweildy since in...,0
3,1.0,2019-11-03T20:09:50.616000,Chandru,Is there a built in heap type in clojure or ho...,0
4,1.0,2019-11-03T20:10:28.069000,Chandru,Just like in any other language (i.e. arraybas...,0
5,3.0,2019-11-03T21:35:02.921000,Jaxiel,... Isn't that basically just a list?,0
...,...,...,...,...,...
1573,34.0,2020-10-13T07:25:41.994000,Crescent,I think you need to use raw sql for that,0
1574,34.0,2020-10-13T09:06:05.440000,Wolfram,yes with hugsql/dbrun I managed to set and upd...,0
1575,34.0,2020-10-13T09:07:03.108000,Wolfram,do you know if there is an intrinsic limitatio...,0
1576,34.0,2020-10-13T11:10:53.300000,Crescent,i dont know really i use honeysql personally b...,-1


In [37]:
# df_sentiCR_racket = pd.read_csv('df_senti4SD_racket.csv', delimiter=';', index_col="ID")
# pred_racket_sentiCR = predict_sentiCR(df_sentiCR_racket, 'Text')
# df_sentiCR_racket['sentiCR'] = Extract(pred_racket_sentiCR)
# df_sentiCR_racket['sentiCR'] = df_sentiCR_racket['sentiCR'].replace({2: -1})
# df_sentiCR_racket.to_csv('predictions_sentiCR_racket.csv')
# df_sentiCR_racket
df_sentiCR_racket = pd.read_csv('predictions_sentiCR_racket.csv', index_col='ID')
df_full_racket = pd.read_csv('full_racket_df.csv', index_col='Unnamed: 0')
df_full_racket['sentiCR'] = df_sentiCR_racket['sentiCR']
df_full_racket

,conversation_id,ts,user,text,sentiCR
1,1.0,2019-11-02T04:46:55.839000,Yiqun,Game Jam has started! https://itch.io/jam/rack...,0
2,2.0,2019-11-02T15:49:08.880000,Yanelly,@notjack @Deleted User `Multiple modules can b...,-1
3,1.0,2019-11-03T09:40:25.704000,Amri,Honestly I'm so proud that I remembered all th...,-1
4,1.0,2019-11-03T10:18:39.088000,Amri,@notjack for when you're next around. Will try...,0
5,1.0,2019-11-03T10:19:10.123000,Amri,gotta love macros xD,0
...,...,...,...,...,...
4971,197.0,2020-10-30T12:58:07.458000,Daevion,(It certainly has technical challenges),0
4972,197.0,2020-10-30T12:58:54.492000,Daevion,Can I suggest an alternative? Set up a github ...,0
4973,197.0,2020-10-30T13:12:33.546000,Abdenacer,oh that's cool idea,1
4974,197.0,2020-10-30T15:44:56.440000,Daevion,I can't recall one. would be fun. 2htdp/univer...,-1


In [38]:
# df_sentiCR_python = pd.read_csv('df_senti4SD_python.csv', delimiter=';', index_col="ID")
# pred_python_sentiCR = predict_sentiCR(df_sentiCR_python, 'Text')
# df_sentiCR_python['sentiCR'] = Extract(pred_python_sentiCR)
# df_sentiCR_python['sentiCR'] = df_sentiCR_python['sentiCR'].replace({2: -1})
# df_sentiCR_python.to_csv('predictions_sentiCR_python.csv')
# df_sentiCR_python
df_sentiCR_python = pd.read_csv('predictions_sentiCR_python.csv', index_col='ID')
df_full_python = pd.read_csv('full_python_df.csv', index_col='Unnamed: 0')
df_full_python['sentiCR'] = df_sentiCR_python['sentiCR']
df_full_python

,conversation_id,ts,user,text,sentiCR
1,1.0,2019-11-01T00:04:50.546000,Patrice,oh noo the orange server's logo is gone,0
2,1.0,2019-11-01T00:05:40.864000,Patrice,to develop mobile apps:,0
3,1.0,2019-11-01T00:07:38.284000,Patrice,is it better to learn kivy or to learn another...,0
4,1.0,2019-11-01T00:19:39.677000,Patrice,someone?,0
5,1.0,2019-11-01T00:21:10.075000,Luna,Unity is quite good but if you already have a ...,1
...,...,...,...,...,...
1254358,2320.0,2020-10-31T18:29:21.070000,Jacopo,Iterator actually you can call `next` directly...,0
1254359,2320.0,2020-10-31T18:29:25.695000,Kwabena,I could not solve most of my problems so I got...,-1
1254360,2320.0,2020-10-31T18:29:39.551000,Prisca,@marc Yes. It operates based on the `__reverse...,0
1254361,2320.0,2020-10-31T18:29:41.070000,Nohan,i dont use linux main because i do not good to...,-1


In [39]:
# df_sentiCR_golang = pd.read_csv('df_senti4SD_golang.csv', delimiter=';', index_col="ID")
# pred_golang_sentiCR = predict_sentiCR(df_sentiCR_golang, 'Text')
# df_sentiCR_golang['sentiCR'] = Extract(pred_golang_sentiCR)
# df_sentiCR_golang['sentiCR'] = df_sentiCR_golang['sentiCR'].replace({2: -1})
# df_sentiCR_golang.to_csv('predictions_sentiCR_golang.csv')
# df_sentiCR_golang
df_sentiCR_golang = pd.read_csv('predictions_sentiCR_golang.csv', index_col='ID')
df_full_golang = pd.read_csv('full_golang_df.csv', index_col='Unnamed: 0')
df_full_golang['sentiCR'] = df_sentiCR_golang['sentiCR']
df_full_golang

,conversation_id,ts,user,text,sentiCR
1,1.0,2019-11-01T04:49:13.533000,Naif,`2019/11/01 15:41:41 http: panic serving 127.0...,-1
2,1.0,2019-11-01T04:49:23.687000,Naif,"```go_ err = db.Exec(""INSERT INTO users VALUES...",0
3,1.0,2019-11-01T04:53:19.220000,Naif,None,0
4,2.0,2019-11-01T04:58:50.798000,Leo,oh boy what do we have here,0
5,2.0,2019-11-01T04:58:54.100000,Leo,SQL concat?,0
...,...,...,...,...,...
247175,1915.0,2020-09-30T18:06:13.071000,Cristofor,then ill check freenode later,0
247176,1915.0,2020-09-30T18:16:02.925000,Samarie,I unironically prefer IRC over Slack,0
247177,1914.0,2020-09-30T18:24:10.424000,Cristofor,as long as its not in the browser me too,-1
247178,1915.0,2020-09-30T18:24:27.760000,Cristofor,i made my GUI build unchanged for Android! 😄,0


In [40]:
from os.path import exists
sentiCR_path = 'preprocessed_sentiCR.csv'

if exists(sentiCR_path):
    df_sentiCR = pd.read_csv(sentiCR_path)
else:
    pred_full = predict_sentiCR(df, 'text')
    df_sentiCR = df.copy()
    df_sentiCR['sentiCR'] = Extract(pred_full)
    df_sentiCR['sentiCR'] = df_sentiCR['sentiCR'].replace({2: -1})
    df_sentiCR.to_csv(sentiCR_path)

In [41]:
from scipy.stats import pearsonr
import pandas as pd

def calculate_pvalues(df):
    dfcols = pd.DataFrame(columns=df.columns)
    pvalues = dfcols.transpose().join(dfcols, how='outer')
    for r in df.columns:
        for c in df.columns:
            tmp = df[df[r].notnull() & df[c].notnull()]
            pvalues[r][c] = round(pearsonr(tmp[r], tmp[c])[1], 4)
    return pvalues

### Senti4SD preprocessing
For the preprocessing of Senti4SD we do the sentiment score calculation outside this notebook file. Make sure yuo save the csv to predictions_senti4SD.csv

In [25]:
df_senti4SD = df.copy()
df_senti4SD = df_senti4SD.drop(['ts', 'conversation_id', 'user'], axis=1).rename(columns={"text": "Text"})
df.index.name = 'ID'
print('Use this dataframe to feed to the senti4SD repo')
df_senti4SD.to_csv('df_senti4SD.csv',sep=';')
df_senti4SD.head()

Use this dataframe to feed to the senti4SD repo


,Text
ID,
4,{{ with .Data }} also works
5,if .Data is invalid it won't be rendered
6,ah
7,true
8,okay okay thank you


In [7]:
senti4SD = pd.read_csv('predictions_senti4SD.csv').sort_values(by=['ID']).set_index('ID')
senti4SD['PREDICTED'] = senti4SD['PREDICTED'].replace({'negative': -1, 'neutral': 0, 'positive': 1})

In [8]:
senti4SD_df = df.copy()
senti4SD_df['senti4SD'] = senti4SD['PREDICTED'].copy()
senti4SD_df.to_csv('preprocessed_senti4SD.csv')
senti4SD_df.head()

,conversation_id,ts,user,text,senti4SD
ID,,,,,
4,1.0,2020-08-01T00:01:15.595000,Samarie,{{ with .Data }} also works,0
5,1.0,2020-08-01T00:01:22.227000,Samarie,if .Data is invalid it won't be rendered,0
6,1.0,2020-08-01T00:02:50.586000,Cephas,ah,0
7,1.0,2020-08-01T00:02:51.116000,Cephas,true,0
8,1.0,2020-08-01T00:02:55.357000,Cephas,okay okay thank you,1


### Calculating the number of messages per conversation

In [42]:
response_df_golang = df_full_golang['conversation_id'].value_counts().rename_axis('conversation_id').to_frame('nr_messages').sort_index(ascending=True)
response_df_golang.head()

,nr_messages
conversation_id,
1.0,80
2.0,19
3.0,56
4.0,207
5.0,23


# SentiCR correlations
##  Correlation between nr of messages and the sentiment of first message in the conversation

In [10]:
first_message_df = df_full_golang.sort_values(by=['ts']).groupby("conversation_id").first()
count_and_text_df = response_df.join(first_message_df)\
.rename(columns={"text": "first_message", "user": "user_first_message", "ts": "ts_first_message"})

In [11]:
count_and_text_df.head()

,nr_messages,ts_first_message,user_first_message,first_message
conversation_id,,,,
1.0,7,2020-08-01T00:01:15.595000,Samarie,{{ with .Data }} also works
2.0,7,2020-08-01T01:09:01.202000,Reeves,Hwllo
3.0,2,2020-08-01T01:27:32.834000,Treysen,The built in proxy stuff is nice
4.0,85,2020-08-01T02:00:57.155000,Josselin,probably
5.0,1,2020-08-01T03:17:19.489000,Maiana,i've codegened > 600 go files that all wrap in...


In [12]:
print("Predicting sentiment with sentiCR on first message of every conversation")
predictions = predict_sentiCR(count_and_text_df)
df_with_sentiment = count_and_text_df.copy()
df_with_sentiment['sentiCR'] = Extract(predictions)
# Replace 2 with -1 as 2 is negative according to this model
df_with_sentiment['sentiCR'] = df_with_sentiment['sentiCR'].replace({2: -1})
df_with_sentiment.head()

Predicting sentiment with sentiCR on first message of every conversation
Prediction used 4.19 seconds


,nr_messages,ts_first_message,user_first_message,first_message,sentiCR
conversation_id,,,,,
1.0,7,2020-08-01T00:01:15.595000,Samarie,{{ with .Data }} also works,1
2.0,7,2020-08-01T01:09:01.202000,Reeves,Hwllo,0
3.0,2,2020-08-01T01:27:32.834000,Treysen,The built in proxy stuff is nice,0
4.0,85,2020-08-01T02:00:57.155000,Josselin,probably,0
5.0,1,2020-08-01T03:17:19.489000,Maiana,i've codegened > 600 go files that all wrap in...,0


In [75]:
corr=df_with_sentiment['nr_messages'].corr(df_with_sentiment['sentiCR'])
print(f"The correlation between the nr messages inside a conversation \nand the sentiCR score of the first message is {corr}")

The correlation between the nr messages inside a conversation 
and the sentiCR score of the first message is -0.0455387783444905


In [92]:
calculate_pvalues(df_with_sentiment[['nr_messages','sentiCR']])

,nr_messages,sentiCR
nr_messages,0.0,0.0463
sentiCR,0.0463,0.0


In [22]:
df_sentiCR_golang = pd.read_csv('df_senti4SD_golang.csv', delimiter=';', index_col="ID")
pred_sentiCR_golang = pd.read_csv('predictions_sentiCR_golang.csv', index_col="ID")
pred_sentiCR_golang

,Text,sentiCR
ID,,
1,`2019/11/01 15:41:41 http: panic serving 127.0...,-1
2,"```go_ err = db.Exec(""INSERT INTO users VALUES...",0
3,None,0
4,oh boy what do we have here,0
5,SQL concat?,0
...,...,...
247175,then ill check freenode later,0
247176,I unironically prefer IRC over Slack,0
247177,as long as its not in the browser me too,-1


## Correlation between avg sentiment and nr of messages in conversation

In [62]:
avg_senti_df = df_sentiCR.copy()
avg_senti_df = avg_senti_df.groupby('conversation_id').mean()
avg_senti_df['nr_messages'] = response_df
avg_senti_df

,ID,sentiCR,nr_messages
conversation_id,,,
1.0,7.000000,0.000000,7
2.0,14.000000,-0.142857,7
3.0,18.500000,0.000000,2
4.0,63.529412,-0.094118,85
5.0,33.000000,0.000000,1
...,...,...,...
1911.0,59922.000000,-1.000000,1
1912.0,60023.500000,0.000000,4
1913.0,60034.500000,-0.083333,12


In [63]:
corr=avg_senti_df['nr_messages'].corr(avg_senti_df['sentiCR'])
corr

-0.013144482517522024

In [79]:
calculate_pvalues(avg_senti_df.drop(['ID'], axis=1))

,sentiCR,nr_messages
sentiCR,0.0,0.5654
nr_messages,0.5654,0.0


# Senti4SD correlations
##  Correlation between nr of messages and the sentiment of first message in the conversation

In [23]:
first_msg_senti4SD = senti4SD_df.sort_values(by=['ts']).groupby("conversation_id").first()
SD_conv_text_df = response_df.join(first_msg_senti4SD)\
.rename(columns={"text": "first_message", "user": "user_first_message", "ts": "ts_first_message"})
SD_conv_text_df.head()

,nr_messages,ts_first_message,user_first_message,first_message,senti4SD
conversation_id,,,,,
1.0,7,2020-08-01T00:01:15.595000,Samarie,{{ with .Data }} also works,0
2.0,7,2020-08-01T01:09:01.202000,Reeves,Hwllo,0
3.0,2,2020-08-01T01:27:32.834000,Treysen,The built in proxy stuff is nice,1
4.0,85,2020-08-01T02:00:57.155000,Josselin,probably,0
5.0,1,2020-08-01T03:17:19.489000,Maiana,i've codegened > 600 go files that all wrap in...,-1


In [24]:
corr=SD_conv_text_df['nr_messages'].corr(SD_conv_text_df['senti4SD'])
print(f"The correlation between the nr messages inside a conversation \nand the senti4SD score of the first message is {corr}")

The correlation between the nr messages inside a conversation 
and the senti4SD score of the first message is -0.02342402019049213


In [91]:
calculate_pvalues(SD_conv_text_df[['nr_messages','senti4SD']])

,nr_messages,senti4SD
nr_messages,0.0,0.3056
senti4SD,0.3056,0.0


## Correlation between avg sentiment and nr of messages in conversation

In [64]:
avg_4sd_df = senti4SD_df.copy()
avg_4sd_df = avg_4sd_df.groupby('conversation_id').mean()
avg_4sd_df['nr_messages'] = response_df
avg_4sd_df.head()

,senti4SD,nr_messages
conversation_id,,
1.0,0.285714,7
2.0,0.142857,7
3.0,1.000000,2
4.0,0.023529,85
5.0,-1.000000,1


In [65]:
corr=avg_4sd_df['nr_messages'].corr(avg_4sd_df['senti4SD'])
print(f"The correlation between the average senti4SD and the number of messages in the conversation is {corr}")

The correlation between the average senti4SD and the number of messages in the conversation is -0.020964833514625834


In [69]:
calculate_pvalues(avg_4sd_df)

,senti4SD,nr_messages
senti4SD,0.0,0.3592
nr_messages,0.3592,0.0


# Per question

In [30]:
questions = df_sentiCR["text"].str.contains("?", regex=False).astype(int)
print("Showing only questions within the sentiCR df:")
df_sentiCR_questions = df_sentiCR[questions == 1]
df_sentiCR_questions.head()

Showing only questions within the sentiCR df:


,ID,conversation_id,ts,user,text,sentiCR
9,13,2.0,2020-08-01T01:09:29.986000,Reeves,I am finding imposible chaining proxies with n...,0
29,33,5.0,2020-08-01T03:17:19.489000,Maiana,i've codegened > 600 go files that all wrap in...,0
47,51,4.0,2020-08-01T03:40:44.702000,Maiana,@_diamondburned_ do you think squeezing the go...,0
48,52,6.0,2020-08-01T03:41:52.495000,Cephas,im displaying this in a HTML where there\'s a ...,0
49,53,4.0,2020-08-01T03:42:55.236000,Cephas,(the struct has no ID and some data pointers a...,0


In [58]:
response_df_reset = response_df.reset_index()
# df_sentiCR_questions.join(response_df_reset, on='conversation_id')
question_sentiCR_df = df_sentiCR_questions.merge(response_df_reset, how='inner', on='conversation_id')

In [59]:
corr=question_sentiCR_df['nr_messages'].corr(question_sentiCR_df['sentiCR'])
print(f"The correlation between a questions sentiCR and the number of messages in the conversation is {corr}")

The correlation between a questions sentiCR and the number of messages in the conversation is 0.013647948199097187


In [89]:
calculate_pvalues(question_sentiCR_df[['nr_messages','sentiCR']])

,nr_messages,sentiCR
nr_messages,0.0,0.255
sentiCR,0.255,0.0


In [7]:
cur_train=pd.read_pickle('SentiCR/data/so-train.pkl')
cur_train[cur_train['label'] == -1]

,sentence,label
8,But the above code doesn't work outside the ``...,-1
9,"When I run the project, I get the following er...",-1
20,"Well, the funny part is, we do not know.",-1
27,I wrote this code to update a patient in a der...,-1
36,There is a caveat with the above approach.,-1
...,...,...
1002,It might not reproduce consistently but at one...,-1
1020,Still that does not guarantee that the first t...,-1
1033,The data structure you are saving your data is...,-1
1037,First of all I do not think you need all these.,-1


In [8]:
cur_train[cur_train['label'] == 0]

,sentence,label
0,Please help me if any changes can be made to t...,0
1,Add public ``getter ''methods where necessary.,0
2,What I am trying to do.,0
3,I have integrated sonarQube for code analysis.,0
4,url WEB_ADDRESS.,0
...,...,...
1044,Just delete the instance variable CODE_FRAGMENT.,0
1045,Store them in memory in a CODE_FRAGMENT.,0
1046,"For example, 8 PM in Auckland NZ is not at all...",0
1047,The line that uses the CODE_FRAGMENT method sh...,0


In [9]:
cur_train[cur_train['label'] == 1]

,sentence,label
30,When I print the path of the resource using AP...,1
33,This is not particularly unusual... or wrong.,1
43,This will preserve the insertion order and ins...,1
51,The simplest solution would be to use Java 8's...,1
63,"For example, the following works great.",1
...,...,...
1008,I understand the boolean expression here.,1
1029,This might prove to be useful unbescape.,1
1036,"John Spurlock, the create of OData4j, personal...",1
1038,Working Configuration.,1
